In [1]:
# see what already avail and thus determine which steps required prior to reading in file and handling the data
# if you see more than "sample_data" you can jump to the relevant step below
!ls

'10a. Journey Data Extract 14Sep14-27Sep14.csv'
'10b. Journey Data Extract 28Sep14-11Oct14.csv'
'11a. Journey Data Extract 12Oct14-08Nov14.csv'
'11b. Journey Data Extract 12Oct14-08Nov14.csv'
'12a. Journey Data Extract 09Nov14-06Dec14.csv'
'12b. Journey Data Extract 09Nov14-06Dec14.csv'
'13a. Journey Data Extract 07Dec14-21Dec14.csv'
'13b. Journey Data Extract 22Dec14-03Jan15.csv'
'14. Journey Data Extract 08Dec13-04Jan14.csv'
'1. Journey Data Extract 05Jan14-02Feb14.csv'
'2. Journey Data Extract 03Feb14-01Mar14.csv'
'3. Journey Data Extract 02Mar14-31Mar14.csv'
'4. Journey Data Extract 01Apr14-26Apr14.csv'
'5. Journey Data Extract 27Apr14-24May14.csv'
'6. Journey Data Extract 25May14-21Jun14.csv'
'7. Journey Data Extract 22Jun14-19Jul14.csv'
'8a Journey Data Extract 20Jul14-31Jul14.csv'
'8b Journey Data Extract 01Aug14-16Aug14.csv'
'9a Journey Data Extract 17Aug14-31Aug14.csv'
'9b Journey Data Extract 01Sep14-13Sep14.csv'
 cyclehireusagestats-2014.zip
 cyclehireusagestats-2014.zip.1
 

In [2]:
# set-up spark (NB if Apache amend versions on download site we will need to amend path in wget command)
print("\nWelcome to advanced top sites")
!ls
!rm -f spark-3.3.[01]-bin-hadoop3.tgz* 
!rm -rf spark-3.3.[01]-bin-hadoop3
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar -xf spark-3.3.2-bin-hadoop3.tgz
!ls -alt



Welcome to advanced top sites
'10a. Journey Data Extract 14Sep14-27Sep14.csv'
'10b. Journey Data Extract 28Sep14-11Oct14.csv'
'11a. Journey Data Extract 12Oct14-08Nov14.csv'
'11b. Journey Data Extract 12Oct14-08Nov14.csv'
'12a. Journey Data Extract 09Nov14-06Dec14.csv'
'12b. Journey Data Extract 09Nov14-06Dec14.csv'
'13a. Journey Data Extract 07Dec14-21Dec14.csv'
'13b. Journey Data Extract 22Dec14-03Jan15.csv'
'14. Journey Data Extract 08Dec13-04Jan14.csv'
'1. Journey Data Extract 05Jan14-02Feb14.csv'
'2. Journey Data Extract 03Feb14-01Mar14.csv'
'3. Journey Data Extract 02Mar14-31Mar14.csv'
'4. Journey Data Extract 01Apr14-26Apr14.csv'
'5. Journey Data Extract 27Apr14-24May14.csv'
'6. Journey Data Extract 25May14-21Jun14.csv'
'7. Journey Data Extract 22Jun14-19Jul14.csv'
'8a Journey Data Extract 20Jul14-31Jul14.csv'
'8b Journey Data Extract 01Aug14-16Aug14.csv'
'9a Journey Data Extract 17Aug14-31Aug14.csv'
'9b Journey Data Extract 01Sep14-13Sep14.csv'
 cyclehireusagestats-2014.zip
 c

In [3]:
# install findspark if not already installed
!pip3 install findspark



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
# init spark (ensure SPARK_HOME set to same version as we download earlier)
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/apache-spark"
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
# the next line gives us 'local' mode. try 'local[2]' to use 2 cores or 'master:NNNN' to run on Spark standalone cluster at port NNNN
spark_conf = SparkConf().setMaster('local[2]').setAppName('MyApp')
sc = SparkContext(conf=spark_conf)
# see what we have by examining the Spark User Interface
from pyspark.sql import *
from pyspark.sql.functions import *
# "SparkSession" and "sc" are are key handles in to Spark API
##SparkSession.builder.getOrCreate()
spark = SparkSession.builder.appName("bikes").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/24 16:03:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
### # OPTIONAL upload any given file using Google Colab API/GUI
### from google.colab import files
### files.upload()


In [6]:
# get file for given year from TfL open data
# !wget https://cycling.data.tfl.gov.uk/usage-stats/cyclehireusagestats-2014.zip
# !unzip cyclehireusagestats-2014.zip



In [7]:
# TO DO: add the relevant spark commands (see week5?)
# read file in to a dataframe called "j_df"
file="./1. Journey*csv"
j_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load(file))


In [8]:

# show top 10 - if this gives an error you need to check the previous step/s
j_df.show(10)
print("duration is SECONDS")
# see how many entries (rows) in data
numRows = j_df.count()
print("there are ",numRows," rows")

# get "list" of last 2 rows
j_df.tail(2)
# we can deduce there is no inherent ordering of the rows

+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+
|Rental Id|Duration|Bike Id|        End Date|EndStation Id|     EndStation Name|      Start Date|StartStation Id|   StartStation Name|
+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+
| 29747123|    1140|    469|05/01/2014 15:54|           55|Finsbury Circus, ...|05/01/2014 15:35|            295|Swan Street, The ...|
| 29764212|    1560|   6924|06/01/2014 23:30|           93|Cloudesley Road, ...|06/01/2014 23:04|            311|Foley Street, Fit...|
| 29824742|    1200|   4115|09/01/2014 23:25|          697|Charlotte Terrace...|09/01/2014 23:05|             81|Great Titchfield ...|
| 29966133|    1380|   2077|17/01/2014 14:06|          695|Islington Green, ...|17/01/2014 13:43|            311|Foley Street, Fit...|
| 29992354|    1320|  12069|18/01/2014 21:06|          

[Row(Rental Id=29790407, Duration=1080, Bike Id=5726, End Date='08/01/2014 09:50', EndStation Id=237, EndStation Name='Vaughan Way, Wapping', Start Date='08/01/2014 09:32', StartStation Id=374, StartStation Name='Waterloo Station 1, Waterloo'),
 Row(Rental Id=29776025, Duration=1500, Bike Id=895, End Date='07/01/2014 16:51', EndStation Id=335, EndStation Name='Tavistock Street, Covent Garden', Start Date='07/01/2014 16:26', StartStation Id=486, StartStation Name='Granby Street, Shoreditch')]

Group and Sort

In [9]:
# group "StartStation Id" and sum their "Duration", sort in descending order
start_df = (j_df.select("StartStation Id", "StartStation Name", "EndStation Id", "EndStation Name","Duration")
         .groupBy("StartStation Name")).sum("Duration").orderBy("sum(Duration)", ascending=False)
# show top 10
start_df.show(10)

+--------------------+-------------+
|   StartStation Name|sum(Duration)|
+--------------------+-------------+
|Hyde Park Corner,...|      8692560|
|Waterloo Station ...|      4865400|
|Albert Gate, Hyde...|      4572960|
|Belgrove Street ,...|      4550760|
|Black Lion Gate, ...|      4208160|
|Westfield Library...|      4192860|
|Speakers' Corner ...|      3508680|
|Whitehall Place, ...|      3461400|
|Sun Street, Liver...|      3256500|
|Queen's Circus, B...|      3218280|
+--------------------+-------------+
only showing top 10 rows



In [10]:
# TO DO: group by ending destination, sum the ride times and sort

Conversions

In [11]:
# more useful perhaps than seconds would be converting to minutes
j_df.show(3)
j_df.selectExpr("Duration/60").show(3) # note that we can show without assigning to a new dataframe

# we can also use 'alias' to give more meaningful column name (note use of "col" from SQL syntax)
j_df.select((col("Duration")/60).alias('mins')).show(3)

min_df=j_df.select("StartStation Id", "StartStation Name", "EndStation Id", "EndStation Name",
                   ((col("Duration")/60).alias('minutes'))
)
min_df.groupBy("StartStation Id", "StartStation Name").sum("minutes").orderBy("sum(minutes)", ascending=False).show(5)

# NOTE that we can expect minutes to be integers since no seconds recorded in original spreadsheet

+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+
|Rental Id|Duration|Bike Id|        End Date|EndStation Id|     EndStation Name|      Start Date|StartStation Id|   StartStation Name|
+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+
| 29747123|    1140|    469|05/01/2014 15:54|           55|Finsbury Circus, ...|05/01/2014 15:35|            295|Swan Street, The ...|
| 29764212|    1560|   6924|06/01/2014 23:30|           93|Cloudesley Road, ...|06/01/2014 23:04|            311|Foley Street, Fit...|
| 29824742|    1200|   4115|09/01/2014 23:25|          697|Charlotte Terrace...|09/01/2014 23:05|             81|Great Titchfield ...|
+---------+--------+-------+----------------+-------------+--------------------+----------------+---------------+--------------------+
only showing top 3 rows

+---------------+
|(Duration /

+---------------+--------------------+------------+
|StartStation Id|   StartStation Name|sum(minutes)|
+---------------+--------------------+------------+
|            191|Hyde Park Corner,...|    144876.0|
|            154|Waterloo Station ...|     81090.0|
|            303|Albert Gate, Hyde...|     76216.0|
|             14|Belgrove Street ,...|     75846.0|
|            307|Black Lion Gate, ...|     70136.0|
+---------------+--------------------+------------+
only showing top 5 rows



In [12]:
# TO DO: group by destination and show total ride duration in hours

Multiple Input Files

In [13]:
# MULTIPLE FILES INPUT AND ANALYSED (presuming identical schema - should test first!)
file1="./1. Journey*csv" # 05 Jan to 02 Feb
file2="./2. Journey*csv" # 03 Feb to 01 Mar
file3="./3. Journey*csv" # 02 Mar to 31 Mar
# we might say therefore that data in these 3 files corresponds to "winter" in the UK
winter_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load([file1, file2, file3])) # i.e pass a Python list of files to load (into a single DF)

In [14]:
# TO DO: how many rows do we have in the winter_df DF?

In [15]:
# TO DO: group by "StartStation Id" and sum their "Duration", sorted with largest number listed first


Aggregates

In [16]:
# Find stats regarding duration of cycle rides 
# TO DO: print out the average and maximum values of "Duration" for winter_df
print("Stats of rides Jan-Mar inclusive (in seconds)")


Stats of rides Jan-Mar inclusive (in seconds)


In [17]:
# Find stats regarding duration of cycle rides grouped by starting destination AND sorted 
print("Sorted grouped-by stats of rides Jan-Mar inclusive")
winter_df.groupBy("StartStation Name").agg({"Duration": "mean"}).orderBy("avg(Duration)", ascending=False).show(8)
winter_df.groupBy("StartStation Name").agg({"Duration": "max"}).sort("max(Duration)", ascending=False).show(8)
# using intermediary DF:
winterSums_df = winter_df.groupBy("StartStation Name").agg({"Duration": "sum"})
winterSums_df.sort("sum(Duration)", ascending=False).show(8)

# TO DO: look at the minimum values for sorted grouped-by stats & at the smallest values of total Durations: what can you conclude?

Sorted grouped-by stats of rides Jan-Mar inclusive


+--------------------+-----------------+
|   StartStation Name|    avg(Duration)|
+--------------------+-----------------+
|Putney Bridge Roa...|5810.224032586558|
|Westfield Library...|5318.042168674699|
|Colet Gardens, Ha...|5282.752293577982|
|Hortensia Road, W...|4742.044653349001|
|Blythe Road West,...|4486.760280842527|
|Lightermans Road,...|4358.747763864043|
|Stanley Grove, Ba...|3979.864864864865|
|Culvert Road, Bat...|3943.601694915254|
+--------------------+-----------------+
only showing top 8 rows



+--------------------+-------------+
|   StartStation Name|max(Duration)|
+--------------------+-------------+
|Norton Folgate, L...|      2406000|
|Sidney Street, St...|      1957320|
|Ilchester Gardens...|      1882860|
|Waterloo Station ...|      1766820|
|Somerset House, S...|      1687620|
|Houghton Street, ...|      1634220|
|Bermondsey Street...|      1624020|
|Charlotte Terrace...|      1576320|
+--------------------+-------------+
only showing top 8 rows



+--------------------+-------------+
|   StartStation Name|sum(Duration)|
+--------------------+-------------+
|Hyde Park Corner,...|     38277180|
|Speakers' Corner ...|     36657300|
|Albert Gate, Hyde...|     22573320|
|Black Lion Gate, ...|     22433100|
|Belgrove Street ,...|     17399040|
|Wellington Arch, ...|     16586040|
|Waterloo Station ...|     15370920|
|Regent's Row , Ha...|     12787380|
+--------------------+-------------+
only showing top 8 rows



Using a "join"

In [18]:
# multiple aggregate columns for grouped by
max_df = winter_df.groupBy("StartStation Name").agg({"Duration":"max"})
min_df = winter_df.groupBy("StartStation Name").agg({"Duration":"min"})

#max_df.show()
#min_df.show()

# join (via use of intermediary DF)
new_df = min_df.join(max_df, on=["StartStation Name"], how="inner")
new_df.show()


+--------------------+-------------+-------------+
|   StartStation Name|min(Duration)|max(Duration)|
+--------------------+-------------+-------------+
|Dunston Road , Ha...|            0|       487740|
|Panton Street, We...|            0|       236520|
|George Place Mews...|            0|       182220|
|Portland Place, M...|            0|        88980|
|  Park Lane, Mayfair|           60|        67620|
|Russell Gardens, ...|            0|        19860|
|Montgomery Square...|            0|       222300|
|Appold Street, Li...|            0|       185400|
|Kingsway Southbou...|            0|        75000|
|Gloucester Street...|            0|       159060|
|Embankment (Horse...|            0|       280020|
|Charlotte Street,...|            0|        70260|
|Lambeth Road, Vau...|            0|        40500|
|Cadogan Close, Vi...|          180|       408420|
|London Zoo, Regen...|            0|       167100|
|Christopher Stree...|            0|      1283520|
|Falcon Road, Clap...|         

In [19]:
# do more aggregate cols and sort by sum(Duration)
mean_df = winter_df.groupBy("StartStation Name").agg({"Duration":"mean"})
sum_df = winter_df.groupBy("StartStation Name").agg({"Duration":"sum"})
new_df = min_df.join(max_df, on=["StartStation Name"], how="inner").join(mean_df, on=["StartStation Name"], how="inner").join(sum_df, on=["StartStation Name"], how="inner").sort("sum(Duration)", ascending=False).show()

+--------------------+-------------+-------------+------------------+-------------+
|   StartStation Name|min(Duration)|max(Duration)|     avg(Duration)|sum(Duration)|
+--------------------+-------------+-------------+------------------+-------------+
|Hyde Park Corner,...|            0|      1136220| 2716.234743116662|     38277180|
|Speakers' Corner ...|            0|       862980|3833.6435892072786|     36657300|
|Albert Gate, Hyde...|            0|       860340| 2230.344827586207|     22573320|
|Black Lion Gate, ...|            0|       408120|2218.2438445565112|     22433100|
|Belgrove Street ,...|            0|       181860| 957.5169225689285|     17399040|
|Wellington Arch, ...|            0|      1044720|2465.2259215219974|     16586040|
|Waterloo Station ...|            0|       110820| 858.2311557788945|     15370920|
|Regent's Row , Ha...|            0|       668340| 1911.702795634624|     12787380|
|Cumberland Gate, ...|            0|      1485540|2988.4491228070174|     12

In [20]:
print("exercise completed. please consider how you would use Spark to answer the following research questions")

exercise completed. please consider how you would use Spark to answer the following research questions


# Research Hypothesis (example 1)
We might state out thinking that "more riding happens in summer than winter". To form a hypothesis, we need to be more precise:

Steps
*   define winter & summer (JFM, JJA say?)
*   define "riding" (sum of duration of rides in given time period); do we want to exclude any rides?
*   if using TfL2014 data, we are only testing for London in that year

Now we can state our hypothesis, which we test whether it is true or not:
"During 2014, more riding happens in London in Summer than Winter, where Summer is defined as JJA and Winter as JFM". And we can state our test for whether this is true: "We define the quantity of riding as the sum of the duration of the relevant rides, and will use TfL 2014 data (citation) to determine if the hypothesis holds for 2014."

We then perform the test by
*   find sum of duration of all valid rides for each period
*   Hypothesis is T (for a given year) if sum of all valid rides in summer exceeds sum of all valid rides in winter for that year.

The hypothesis is perhaps too narrow so you might want to think how to expand to not just London in 2014.


# Research Hypothesis (example 2)
Test the hypothesis "people ride for longer in June than in January". 

Possible outline steps
*   find mean of journeys in Jan
*   find mean of journeys in June
*   Hypothesis is T if mean of June > mean of Jan



